---

# Phase 3. Data Preparation
The data preparation phase covers all activities needed to construct the final dataset [data that will be fed into the modeling tool(s)] from the initial raw data. Data preparation tasks are likely to be performed multiple times and not in any prescribed order. Tasks include table, record, and attribute selection, as well as transformation and cleaning of data for modeling tools

---

## 3.1 Select data

### 3.1.1 Task
Decide on the data to be used for analysis.

#### Cash Outlier Modelling Data
- Customer Geographic Data
- Branch Geographic Data
- Turnover Data
- Cash Transactions Data

#### Domestic Outlier Modelling Data
- Customer Geographic Data
- Branch Geographic Data
- Turnover Data
- Domestic Transactions Data


### 3.1.2 Output
#### 3.1.2.1 Rationale for inclusion/exclusion
List the data to be included/excluded and the reasons for these decisions.

Data for inclusion
- a
- b
- c

Data to be excluded
- d
- e
- f

---

## 3.2 Clean data

### 3.2.1 Task

Raise the data quality to the level required by the selected analysis techniques.

### 3.2.1.1 Cleaning Task: Rename column 'customer_source_unique_id' to 'cust_id'

In [18]:
df_turnovers.rename(columns={'customer_source_unique_id':'cust_id'}, inplace=True)

### 3.2.2 Output

#### 3.2.2.1 Data cleaning report

Describe what decisions and actions were taken to address any data quality problems reported
during the Verify Data Quality task of the Data Understanding phase.

---

## 3.3 Construct data

### 3.3.1 Task

This task includes constructive data preparation operations such as the production of derived
attributes or entire new records, or transformed values for existing attributes.

#### 3.3.1.1 Construct Task: Setup helper Functions
Establish some helper functions to simplify the code later in the process.

In [48]:
#haversine dist 
def haversine(x): 
    lat1, lon1, lat2, lon2 = map(radians, [x.latitude, x.longitude, x.cust_lat, x.cust_long])
    dlon = lon2-lon1 
    dlat = lat2-lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2 
    c = 2*asin(min(sqrt(a),1))
    return c * 6371

In [49]:
def interquartile_range(x): 
    return x.quanti1e(.75) - x.quanti1e(.25)

In [20]:
def check_account_number(counter_party_account_num):
    
    sort_code = None
    account_number = None
    num_characters_after_space = 0
    account_number_correct_format = "N"
    split_string = []

    counter_party_account_num_str = str(counter_party_account_num)
    # Check if the account number contains any non-numeric characters.
    if not re.match(r"\d+", counter_party_account_num_str):
    #returning tuple of sort_code, account_number, account_number_correct_format
        return sort_code, account_number, account_number_correct_format
        
    if " " in counter_party_account_num_str:
        split_string = counter_party_account_num_str.split(" ")
        characters_before_space = split_string[0]
        characters_after_space = split_string[1]
        num_characters_before_space = len(characters_before_space)
        num_characters_after_space = len(characters_after_space)
    else:
        num_characters_before_space = len(counter_party_account_num_str)
        characters_before_space = counter_party_account_num_str
    
    # Future Plan to verify sort code against EISCD

    if num_characters_before_space == 6:
        sort_code = characters_before_space
    elif num_characters_after_space == 6:
        sort_code = characters_after_space
    
    if num_characters_before_space == 8:
        account_number = characters_before_space
    elif num_characters_after_space == 8:
        account_number = characters_after_space
    
    if sort_code is not None and account_number is not None:
        account_number_correct_format = "Y"
    elif sort_code is not None or account_number is not None:
        account_number_correct_format = "N"
    else:
        sort_code = None
        account_number = None
        account_number_correct_format = "N"

    #returning tuple of sort_code, account_number, account_number_correct_format
    return sort_code, account_number, account_number_correct_format

In [44]:
all_months={'202105', '202106', '202107', '202108', '202109', '202110', '202111', '202112',
           '202201', '202202', '202203', '202204', '202205', '202206', '202207', 
           '202208', '202209', '202210', '202211', '202212', '202301', '202302',
           '202303', '202304', '202305', '202306', '202307', '202308'}

def add_empty_months(monthly_in, all_months, jnumber):
    new_df=monthly_in.copy()
    new_df['year_month']=new_df['year'].astype(str)+new_df['month'].astype(str).apply(lambda x: ('0'+x)[-2:])

    new_records=[]
    for year_month in all_months-set(list(new_df['year_month'])):
        new_record={'customer_source_unique_id': jnumber, 'year': int(year_month[:4]), 'month': int(year_month[4:]), 'sum_in': 0,
                   'average_in': np.nan, 'median_in': np.nan, 'max_in': np.nan, 'no_deposits_in': 0,
                   'variance_in': np.nan, 'iqr_in': np.nan, 'year_month': year_month}
        new_records.append(new_record)
    new_df=pd.concat([new_df,pd.DataFrame(new_records)])
    new_df=new_df.sort_values('year_month')
    new_df.reset_index(inplace=True)
    new_df.drop('year_month', axis=1, inplace=True)
    return new_df

#### 3.3.1.2 Construct Task: add df_turnovers attributes

In [21]:
df_turnovers['credit_2y'] = df_turnovers['credit_turnover_12'] + df_turnovers['credit_turnover_24'] 
df_turnovers['debit_2y'] = df_turnovers['debit_turnover_12'] + df_turnovers['debit_turnover_24']  
df_turnovers['credit_yoy'] = df_turnovers['credit_turnover_12'] / df_turnovers['credit_turnover_24']
df_turnovers['debit_yoy'] = df_turnovers['debit_turnover_12'] / df_turnovers['debit_turnover_24']

#### 3.3.1.3 Construct Task: Cash Transactions Import Function

What does the task/function do

- filter by "cash" instrument
- convert to 'txn_amount_base' to float
- expand posting_date (break out month and day, assign a period classification (12 month or 24 month) based on date range)
- assign 'txn_amount_base' to either 'txn_val_24months' or 'txn_val_12months'
- fill empty 'transaction_location' values
- if a transaction items 'reversal' value is 'yes' then multiply 'txn_amount_base' by -1.0 
- extract 'centre_code' from subsection of 'source_txn_unique_id'
- if 'instrument' = 'CASH - CHANNELS' or 'cash - channels_rv' assign 'Y' to 'PO' column

What does it return

In [22]:
def read_clean(folder, file) : 
    df = pd.read_csv(os.path.join(folder,file)) 
    jnumber = df['customer_source_unique_id'].iloc[0]
    df = df[['customer_source_unique_id', 'in_out', 'posting_date', 'txn_amount_base', 'instrument', 'region', 'reversal', \
             'transaction_location', 'transaction_location_cntry_cd', 'source_txn_unique_id']]
    # filter by "cash" instrument
    df = df[(df['instrument'].str.contains('CASH')) & (df.transaction_location_cntry_cd.isin([np.nan, 'NULL', '[NULL]', 'GB']))] #Domestic cash only 

    # convert to 'txn_amount_base' to float
    df['txn_amount_base'] = df['txn_amount_base'].astype(float)

    # expand posting_date (break out month and day, assign a period classification (12 month or 24 month) based on date range)
    df['date'] = df['posting_date']. apply(lambda x: datetime.strptime(str(x), '%Y%m%d')) 
    df['month'] = df['date'].apply(lambda x: x.month)
    df['day'] = df['date'].apply(lambda x: x.day)
    df['period'] = df.posting_date.apply(lambda x: '12 month' if (x>20220520 and x <= 20230520) \
                                         else '24 month' if x<= 20220520 else 'Later')
    
    # assign 'txn_amount_base' to either 'txn_val_24months' or 'txn_val_12months'
    # if df empty pass in values from 'txn_amount_base'
    if len(df) == 0:
        df['txn_val_24months'] = df['txn_amount_base']
        df['txn_val_12months'] = df['txn_amount_base'] 
    else:
        df['txn_val_24months'] = df.apply(lambda x: x['txn_amount_base'] if x['period'].lower() == '24 month' else np.nan, axis=1)
        df['txn_val_12months'] = df.apply(lambda x: x['txn_amount_base'] if x['period'].lower() == '12 month' else np.nan, axis=1)

        # fill empty 'transaction_location' values
        df['transaction_location'] = df['transaction_location'].replace(np.nan, '')

        # if a transaction items 'reversal' value is 'yes' then multiply 'txn_amount_base' by -1.0 
        df['txn_amount_base'] = [x if y == 'N' else x*-1.0 for x,y in zip(df['txn_amount_base'], df['reversal'])]

        # extract 'centre_code' from subsection of 'source_txn_unique_id'
        df['centre_code'] = df['source_txn_unique_id'].str[11:15]

        # if 'instrument' = 'CASH - CHANNELS' or 'cash - channels_rv' assign 'Y' to 'PO' column
        df['PO'] = df['instrument'].map(lambda x: 'Y' if x.lower() == 'CASH - CHANNELS' \
                                        or x.lower() == 'cash - channels_rv' else 'N')

    return df, jnumber

#### 3.3.1.3 Construct Task: Cash Basic Statistics Function
- statistical calculations for both 'cash in' and 'cash out'
- dataframe for statistical calculation results "result_in" and "result_out"
- dataframe for 'txn_val_12months', 'txn_val_24months', 'cash_in_ratio' and 'cash_out_ratio'

In [23]:
# F03, F04, F05, F06, F10, F16
def basic_stats_features(df, jnumber):
    df_in = df.loc[df['in_out'].str.lower() == 'in']
    df_out = df.loc[df['in_out'].str.lower() == 'out']
    if len(df_in) == 0:
        result_in = pd.DataFrame([[jnumber]+list(np.linspace(0,0,10, dtype=int))]\
                                , columns = ['customer_source_unique_id', 'sum_in', 'average_in', 'median_in',
                                'max_in', 'no_deposits_in', 'variance_in', 'iqr_in', 'skewness_in',
                                'kurtosis_in', 'average_cash_in_to_interq'])
        result2 = pd.DataFrame([[jnumber, 0,0,0]]\
                                , columns = ['customer_source_unique_ id', 'txn_val_24months', 'txn_val_12months',
                                'cash_ in_ratio'])
    else:
        result_in = df_in.groupby( 'customer_source_unique_id').agg(
            sum_in = ('txn_amount_base', np.sum),
            average_in = pd.NamedAgg(column='txn_amount_base', aggfunc=lambda x: np.mean([i for i in x if i != 0])),
            median_in = pd.NamedAgg(column='txn_amount_base', aggfunc=lambda x: np.median([i for i in x if i != 0])),
            max_in = ('txn_amount_base', np.max),
            no_deposits_in = ('txn_amount_base', 'count'),
            variance_in = pd.NamedAgg(column='txn_amount_base', aggfunc=lambda x: np.var([i for i in x if i != 0])),
            iqr_in = pd.NamedAgg(column='txn_amount_base', aggfunc=lambda x : iqr([i for i in x if i != 0])),
            skewness_in = pd.NamedAgg(column='txn_amount_base', aggfunc=lambda x: skew([i for i in x if i != 0])),
            kurtosis_in = pd.NamedAgg(column='txn_amount_base', aggfunc=lambda x: kurtosis([i for i in x if i != 0]))
            )
        result_in.reset_index(inplace = True)
        result_in['average_cash_in_to_interq'] = result_in['average_in']/result_in ['iqr_in']

        result2 = df_in.groupby( 'customer_source_unique_id')[['txn_val_24months', 'txn_val_12months']].sum().reset_index()
        result2['cash_in_ratio'] = result2['txn_val_12months']/result2['txn_val_24months']

    if len(df_out) == 0:
        result_out = pd.DataFrame([[jnumber]+list(np.linspace(0,0,9, dtype=int))]\
                                , columns = ['customer_source_unique_id', 'sum_out', 'average_out', 'median_out',
                                'max_out', 'no_deposits_out', 'variance_out', 'iqr_out', 'skewness_out',
                                'kurtosis_out'])
        result2_out = pd.DataFrame([[jnumber, 0 ,0,0]]\
        , columns = ['customer_source_unique_id', 'txn_val_24months', 'txn_val_12months',
        'cash_out_ratio'])
    else:

        result_out = df_out.groupby( 'customer_source_unique_id').agg(
            sum_out = ('txn_amount_base', np.sum),
            average_out = pd.NamedAgg(column='txn_amount_base', aggfunc=lambda x: np.mean([i for i in x if i != 0])),
            median_out = pd.NamedAgg(column= 'txn_amount_base', aggfunc=lambda x: np.median([i for i in x if i != 0])),
            max_out = ('txn_amount_base', np.max),
            no_deposits_out = ('txn_amount_base', 'count'),
            variance_out = pd.NamedAgg(column='txn_amount_base', aggfunc=lambda x: np.var([i for i in x if i != 0])),
            iqr_out = pd.NamedAgg(column='txn_amount_base', aggfunc=lambda x: iqr([i for i in x if i != 0])),
            skewness_out = pd.NamedAgg(column='txn_amount_base', aggfunc=lambda x: skew([i for i in x if i != 0])),
            kurtosis_out = pd.NamedAgg(column='txn_amount_base', aggfunc=lambda x: kurtosis([i for i in x if i != 0]))
            )
        result_out.reset_index(inplace = True)
        result2_out = df_out.groupby( 'customer_source_unique_id')[['txn_val_24months', 'txn_val_12months']] .sum() .reset_index()
        result2_out ['cash_out_ratio'] = result2_out ['txn_val_12months']/result2_out ['txn_val_24months']
    ### drop year on year values
    result2 = result2.merge(result2_out, how = 'outer', on = 'customer_source_unique_id')
    return result_in, result_out, result2


#### 3.3.1.3 Construct Task: Geographic Features Function
- geographic features for 'cash in'
- 
- dataframe for
-- result3
-- result4
-- result8
-- result9

In [24]:
def geographic_features(df, jnumber):
    df_in = df.loc[df['in_out'].str.lower() == 'in']
    # Postcode features
    df_temp = df_in.copy()
    # df_temp = df_temp[ df_temp['transaction_location'] ! = " ]
    # df_temp['sum_in'] = df_temp.groupby(['customer_source_unique_id', 'centre_code'])['txn_amount_base'].transform('sum')
    # df_temp['average_in'] = df_temp.groupby(['customer_source_unique_id', 'centre_code'])['txn_amount_base'].transform('mean')
    # df_temp['no_deposits'] = df_temp.groupby(['customer_source_unique_id','centre_code'])['txn_amount_base'].transform('count')
    # df_temp['total_in'] = df_temp.groupby('customer_source_unique_id')['txn_amcunt_base'].transform('sum')
    df_temp = df_temp.groupby(['customer_source_unique_id', 'instrument', 'centre_code']).agg(
        sum_in = ( 'txn_amount_base', np.sum),
        #mean = pd.NamedAgg(column='txn_amount_base', aggfunc=lambda x: np.mean([i for i in x if i != 0])),
        no_deposits_in = ('txn_amount_base', 'count')
        )
    df_temp['total_in'] = df_temp.groupby( 'customer_source_unique_id')['sum_in'].transform('sum')
    df_temp.reset_index(inplace = True)
    if len(df_temp) == 0:
        result3 = pd.DataFrame({'customer_source_unique_id':[jnumber],
                                'filtered_sum_in': [0],
                                'sum_in ': [0],
                                'PO_CDM_cash_pct': [0],
                                'PO_CDM_cash_only': [0]})
    else:
        df_temp['filtered_sum_in'] = df_temp.apply(
            lambda x: x['sum_in'] if x['instrument'].lower() == 'cash - channels' or x['instrument'].lower() == 'cash - channels_rv' else 0, axis=1)

        result3 = df_temp.groupby('customer_source_unique_id')[['filtered_sum_in','sum_in']].sum().reset_index()
        result3['PO_CDM_cash_pct'] = result3['filtered_sum_in']/result3['sum_in']

        result3['PO_CDM_cash_only'] = result3['PO_CDM_cash_pct'].apply(lambda x: 1 if x == 1.0 else 0)

    result3 = result3[['customer_source_unique_id', 'PO_CDM_cash_pct','PO_CDM_cash_only']]

    df_branch_sig=df_temp.copy()
    df_branch_sig = df_branch_sig[(df_branch_sig['instrument'].isin(['CASH - BRANCH', 'CASH - BRANCH_RV'])) & (df_branch_sig['centre_code'].astype(str) != '1602')]
    df_branch_sig['branch_count'] = df_branch_sig.groupby('customer_source_unique_id')['centre_code'].transform('count')
    df_branch_sig['sig_branch'] = np.where(df_branch_sig['sum_in'] >= (sum(df_branch_sig['sum_in']) * 0.1 / df_branch_sig['branch_count']), 1, 0)

    if len(df_branch_sig) == 0:
        result9 = pd.DataFrame([[jnumber, 0, 0, 0]], columns = ['customer_source_unique_id', \
                                'distance', 'distanceYoY', 'deposit_accessibility'])
        result4 = pd.DataFrame([[jnumber, 0, 0]], columns = ['customer_source_unique_id',\
                                'significant_branches_count', 'average_deposit_sig_branches'])
        result8 = pd.DataFrame([[jnumber, 0]], columns = ['customer_source_unique_id', \
                                'branch_changes_yoy'])
    else:

        # F01 & F02 (PO not included)
        #---------------------------------------------------------


        branch_total = sum(df_branch_sig.sum_in)
        no_sig_branch = sum(df_branch_sig.sig_branch)
        try:
            avg_sig_branch = branch_total/no_sig_branch
        except:
            avg_sig_branch = 0
        
        result4 = pd.DataFrame([[jnumber, no_sig_branch, avg_sig_branch]], columns = ['customer_source_unique_id', 'significant_branches_count', 'average_deposit_sig_branches'])
        
        # Branch accessibility features and YoY branch change
        # Fll, F12.,. f21 & F24
        #---------------------------------------------------------
        df_branch_change = df_in[(df_in['instrument'].isin(['CASH - BRANCH', 'CASH - BRANCH_RV'])) & (df_in['centre_code'].astype(str) != '1602')]
        df_branch_change = df_branch_change[['customer_source_unique_id','period','centre_code']].drop_duplicates()
        df_branch_change = df_branch_change.groupby(['customer_source_unique_id', 'centre_code'] \
                ).count().reset_index().rename(columns={ 'period':'no_years'})
        df_branch_change = df_branch_change[df_branch_change['no_years'] != 2]
        if len(df_branch_change) == 0:
            result8 = pd.DataFrame([[jnumber, 0]], columns = ['customer_source_unique_id',\
                                            'branch_changes_yoy'])
        else:
            result8 = df_branch_change.groupby('customer_source_unique_id')['no_years'].sum().reset_index().rename(columns = {'no_years': 'branch_changes_yoy'})

        df_branch = df_in[(df_in['instrument'].isin(['CASH - BRANCH', 'CASH - BRANCH_RV'])) & (df_in['centre_code'].astype(str) != '1602')]
        df_branch = df_branch.merge(df_branch_11, how='left', on='centre_code')
        
        df_branch12 = df_branch[df_branch['period'] == '12 month']
        df_branch24 = df_branch[df_branch['period'] == '24 month']
        sum_12 = df_branch12.txn_amount_base.sum()
        sum_24 = df_branch24.txn_amount_base.sum()
        
        sum_deposit = df_branch.txn_amount_base.sum()
        df_sum = df_branch[['customer_source_unique_id','latitude','longitude','txn_amount_base']].groupby(['customer_source_unique_id','latitude','longitude']\
                                                            , as_index=False).txn_amount_base.sum()
        df_sum12 = df_branch12[['customer_source_unique_id', 'latitude', 'longitude', 'txn_amount_base']].groupby(['customer_source_unique_id','latitude', 'longitude'] \
                                                            , as_index=False).txn_amount_base.sum()
        df_sum24 = df_branch24[['customer_source_unique_id', 'latitude', 'longitude', 'txn_amount_base']].groupby(['customer_source_unique_id', 'latitude', 'longitude']\
                                                            , as_index=False).txn_amount_base.sum()
        df_sum = df_sum.merge(df_cust_ll, how='left', on='customer_source_unique_id')
        df_sum12 = df_sum12.merge(df_cust_ll, how='left', on='customer_source_unique_id')
        df_sum24 = df_sum24.merge(df_cust_ll, how='left', on='customer_source_unique_id')
        
        
        df_sum['dist'] = df_sum.apply(haversine, axis=1)
        
        df_sum12['dist'] = df_sum12.apply(haversine, axis=1)
        df_sum24['dist'] = df_sum24.apply(haversine, axis=1)
        
        gdl = (df_sum['dist']*df_sum['txn_amount_base']/sum_deposit).sum()
        gdl_12 = (df_sum12['dist']*df_sum12['txn_amount_base']/sum_12).sum()
        gdl_24 = (df_sum24['dist']*df_sum24['txn_amount_base']/sum_24).sum()
        
        try:
            gdlYoY = gdl_12/gdl_24
        except:
            gdlYoY = 0
        
        da = len(df_sum[df_sum.dist <= 10000])
        
        result9 = pd.DataFrame([[jnumber, gdl, gdlYoY, da]], columns = ['customer_source_unique_id',\
                                                                        'distance', 'distanceYoY', 'deposit_accessibility'])
    return result3, result4, result8, result9

In [25]:
def frequency_features(df, jnumber):
    df_in = df.loc[df['in_out'].str.lower() == 'in ']
    # F25 & F26
    #- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
    
    if len(df_in) == 0:
        result5 = pd.DataFrame([[jnumber,0,0]]\
                                , columns = ['customer_source_unique_id', 'var_monthly_sum_in', 'var_monthly_average_in'])
        result6 = pd.DataFrame([[jnumber,0,0,0,0,0]]\
                                , columns = ['customer_source_unique_id', 'depvar', 'mean12', 'mean24', 'mean2y',
                                'meanYoY'])
    else:
        result5 = df_in.groupby(['customer_source_unique_id', 'month', 'period']).agg(
            monthly_sum_in=pd.NamedAgg(column='txn_amount_base', aggfunc=np.sum),
            monthly_average_in=pd.NamedAgg(column='txn_amount_base', aggfunc=lambda x: np.mean([i for i in x if i != 0]))
            )

        result5.reset_index(inplace=True)

        result5 = result5.groupby('customer_source_unique_id').agg(
            var_monthly_sum_in=pd.NamedAgg(column='monthly_sum_in', aggfunc=np.var),
            var_monthly_average_in=pd.NamedAgg(column='monthly_average_in' , aggfunc=np.var)
            )
        
        result5.reset_index(inplace=True)
        
        # F27, F28 & F29
        #- -- - - - - - - - - - - - - - - - - - - - - - - -- - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
        all_months = [1,2,3,4,5,6,7,8,9,10,11,12]
        all_years = ['12 month', '24 month']
        all_jnumbers = df['customer_source_unique_id'].unique()
        
        df_temp_freq = pd.DataFrame([(jnumber, period, month) for jnumber in all_jnumbers\
                for period in all_years for month in all_months], \
                        columns = ['customer_source_unique_id', 'period', 'month'])
        
        df_temp_freq = df_temp_freq.merge(df_in, how='left', on=['customer_source_unique_id', 'period', 'month'])
        df_temp_freq.fillna(value=0, inplace=True)
        df_temp_freq = df_temp_freq[['customer_source_unique_id', 'period', 'month', 'txn_amount_base']]
        
        tl = df_temp_freq.groupby(['customer_source_unique_id', 'period', 'month'])[\
                'txn_amount_base'].agg(no_deposits = lambda x: np.sum([1 for i in x if i != 0]),\
                                        month_tot_cash = 'sum').reset_index().fillna(value = 0)
        
        result6 = tl.groupby( 'customer_source_unique_id')['no_deposits'].agg( depvar = 'var').reset_index()
        
        t2 = tl.groupby(['customer_source_unique_id','period'])['no_deposits'].agg(mean_no_deposits='mean').reset_index()
        # al = t2.pivot(index='customer_source_unique_id', columns = 'period', values=['mean_no_deposits']).reset_index()
        # tl00 = a1.unstack()
        # al.rename(columns = {('12 month':'mean12', '24 month':'mean24'}, inplace = True)
        # al.columns
        mean12 = t2.loc[t2['period'] == '12 month','mean_no_deposits'].iat[0]
        mean24 = t2.loc[t2['period'] == '24 month','mean_no_deposits'].iat[0]
        a1 = pd.DataFrame([(jnumber, mean12, mean24)], columns = ['customer_source_unique_id', 'mean12', 'mean24'])
        result6 = pd.merge(result6, a1, on='customer_source_unique_id', how='left')
        
        t3 = tl.groupby('customer_source_unique_id')['no_deposits'].agg(mean2y = 'mean').reset_index()
        
        result6 = pd.merge(result6, t3, on= 'customer_source_unique_id', how='left')
        result6['meanYoY'] = np.where(result6['mean24']!=0, result6['mean12']/result6['mean24'], 0)
    return result5, result6

In [37]:
def read_domestic_clean(folder, file) : 
    df = pd.read_csv(os.path.join(folder,file)) 
    jnumber = df['customer_source_unique_id'].iloc[0]
    df = df[['customer_source_unique_id', 'in_out', 'posting_date', 'txn_amount_base', 'instrument', 'region', 'reversal', \
             'transaction_location', 'transaction_location_cntry_code', 'source_txn_unique_id', 'counter_party_account_num', \
             'counter_party_name']]
    df = df[df['instrument'].str.contains('CASH', case=False)==False] #exclude cash (also ignoring case sensitivity) 
    
    df['txn_amount_base'] = df['txn_amount_base'].astype(float)
    df['date'] = df['posting_date']. apply(lambda x: datetime.strptime(str(x), '%Y%m%d')) 
    df['year'] = df['date'].apply(lambda x: x.year)
    df['month'] = df['date'].apply(lambda x: x.month)
    df['day'] = df['date'].apply(lambda x: x.day)
    df['period'] = df.posting_date.apply(lambda x: '12 month' if (x>20220520 and x <= 20230520) \
                                         else '24 month' if x<= 20220520 else 'Later')
    if len(df) == 0:
        df['txn_val_24months'] = df['txn_amount_base']
        df['txn_val_12months'] = df['txn_amount_base'] 
    else:
        df['txn_val_24months'] = df.apply(lambda x: x['txn_amount_base'] if x['period'].lower() == '24 month' else np.nan, axis=1)
        df['txn_val_12months'] = df.apply(lambda x: x['txn_amount_base'] if x['period'].lower() == '12 month' else np.nan, axis=1)

        df['transaction_location'] = df['transaction_location'].replace(np.nan, '')
        df['transaction_location_cntry_code'] = df['transaction_location_cntry_code'].replace(np.nan, '')
        df['txn_amount_base'] = [x if y == 'N' else x*-1.0 for x,y in zip(df['txn_amount_base'], df['reversal'])]
        df['centre_code'] = df['source_txn_unique_id'].str[11:15]
        df['counter_party_account_num'] = df['counter_party_account_num'].replace(np.nan, '')
        df['sort_code'],  df['account_number'], df['account_number_correct_format'] = zip(*df['counter_party_account_num'].apply(check_account_number))

    return df, jnumber

In [39]:
# Receipts skewness & kurtosis
def basic_domestic_stats_features(df, jnumber):
    df_in = df.loc[df['in_out'].str.lower() == 'in']
    df_out = df.loc[df['in_out'].str.lower() == 'out']
    if len(df_in) == 0:
        result_in = pd.DataFrame([[jnumber]+list(np.linspace(0,0,10, dtype=int))]\
                                , columns = ['customer_source_unique_id', 'sum_in', 'average_in', 'median_in',
                                'max_in', 'no_deposits_in', 'variance_in', 'iqr_in', 'skewness_in',
                                'kurtosis_in', 'average_receipt_in_to_interq', 'average_receipts_per_month', 
                                'sum_receipts_to_credit_turnover', 'average_receipts_to_credit_turnover'])
        result2 = pd.DataFrame([[jnumber, 0,0,0]]\
                                , columns = ['customer_source_unique_ id', 'txn_val_24months', 'txn_val_12months',
                                'receipt_in_ratio'])
    else:
        result_in = df_in.groupby( 'customer_source_unique_id').agg(
            sum_in = ('txn_amount_base', np.sum),
            average_in = pd.NamedAgg(column='txn_amount_base', aggfunc=lambda x: np.mean([i for i in x if i != 0])),
            median_in = pd.NamedAgg(column='txn_amount_base', aggfunc=lambda x: np.median([i for i in x if i != 0])),
            max_in = ('txn_amount_base', np.max),
            no_deposits_in = ('txn_amount_base', 'count'),
            variance_in = pd.NamedAgg(column='txn_amount_base', aggfunc=lambda x: np.var([i for i in x if i != 0])),
            iqr_in = pd.NamedAgg(column='txn_amount_base', aggfunc=lambda x : iqr([i for i in x if i != 0])),
            skewness_in = pd.NamedAgg(column='txn_amount_base', aggfunc=lambda x: skew([i for i in x if i != 0])),
            kurtosis_in = pd.NamedAgg(column='txn_amount_base', aggfunc=lambda x: kurtosis([i for i in x if i != 0]))
            )
        result_in.reset_index(inplace = True)
        result_in['average_receipt_in_to_interq'] = result_in['average_in']/result_in ['iqr_in']
        result_in['average_receipts_per_month'] = result_in['no_deposits_in']/24
        result_in['sum_receipts_to_credit_turnover'] = result_in['sum_in']/df_turnovers['credit_2y']
        result_in['average_receipts_to_credit_turnover'] = result_in['average_in']/df_turnovers['credit_2y']
        
        result2 = df_in.groupby('customer_source_unique_id')[['txn_val_24months', 'txn_val_12months']].sum().reset_index()
        result2['receipt_in_ratio'] = result2['txn_val_12months']/result2['txn_val_24months']

        #calculated results by instrument type
        result_by_instrument = {} 
        #loop through instrument types
        for instrument_type in df_in['instrument'].unique():
            instrument_type_str = str(instrument_type)
            instrument_df = df_in.loc[df_in['instrument'] == instrument_type_str]
            result_instrument = instrument_df.groupby( 'customer_source_unique_id').agg(
                sum_in = ('txn_amount_base', np.sum),
                average_in = pd.NamedAgg(column='txn_amount_base', aggfunc=lambda x: np.mean([i for i in x if i != 0])),
                median_in = pd.NamedAgg(column='txn_amount_base', aggfunc=lambda x: np.median([i for i in x if i != 0])),
                max_in = ('txn_amount_base', np.max),
                no_deposits_in = ('txn_amount_base', 'count'),
                variance_in = pd.NamedAgg(column='txn_amount_base', aggfunc=lambda x: np.var([i for i in x if i != 0])),
                iqr_in = pd.NamedAgg(column='txn_amount_base', aggfunc=lambda x : iqr([i for i in x if i != 0])),
                skewness_in = pd.NamedAgg(column='txn_amount_base', aggfunc=lambda x: skew([i for i in x if i != 0])),
                kurtosis_in = pd.NamedAgg(column='txn_amount_base', aggfunc=lambda x: kurtosis([i for i in x if i != 0]))
                )
            result_instrument.reset_index(inplace = True)
            result_instrument = result_instrument.assign(instrument_type=instrument_type_str)
            result_instrument['average_receipt_in_to_interq'] = result_instrument['average_in']/result_in ['iqr_in']
            result_instrument['average_receipts_per_month'] = result_instrument['no_deposits_in']/24
            result_instrument['percent_instrument_type_of_total'] = result_instrument['sum_in']/result_in['sum_in']
            result_by_instrument[instrument_type_str] = result_instrument
        result_by_instrument_in = pd.concat(list(result_by_instrument.values()))
    
    if len(df_out) == 0:
        result_out = pd.DataFrame([[jnumber]+list(np.linspace(0,0,9, dtype=int))]\
                                , columns = ['customer_source_unique_id', 'sum_out', 'average_out', 'median_out',
                                'max_out', 'no_deposits_out', 'variance_out', 'iqr_out', 'skewness_out',
                                'kurtosis_out', 'average_payments_out_to_interq', 'average_payments_per_month',
                                'sum_payments_to_debit_turnover', 'average_payments_to_debit_turnover'])
        result2_out = pd.DataFrame([[jnumber, 0 ,0,0]]\
        , columns = ['customer_source_unique_id', 'txn_val_24months', 'txn_val_12months',
        'cash_out_ratio'])
    else:
        result_out = df_out.groupby('customer_source_unique_id').agg(
            sum_out = ('txn_amount_base', np.sum),
            average_out = pd.NamedAgg(column='txn_amount_base', aggfunc=lambda x: np.mean([i for i in x if i != 0])),
            median_out = pd.NamedAgg(column= 'txn_amount_base', aggfunc=lambda x: np.median([i for i in x if i != 0])),
            max_out = ('txn_amount_base', np.max),
            no_deposits_out = ('txn_amount_base', 'count'),
            variance_out = pd.NamedAgg(column='txn_amount_base', aggfunc=lambda x: np.var([i for i in x if i != 0])),
            iqr_out = pd.NamedAgg(column='txn_amount_base', aggfunc=lambda x: iqr([i for i in x if i != 0])),
            skewness_out = pd.NamedAgg(column='txn_amount_base', aggfunc=lambda x: skew([i for i in x if i != 0])),
            kurtosis_out = pd.NamedAgg(column='txn_amount_base', aggfunc=lambda x: kurtosis([i for i in x if i != 0]))
            )
        result_out.reset_index(inplace = True)
        result_out['average_payments_out_to_interq'] = result_out['average_out']/result_out['iqr_out']
        result_out['average_payments_per_month'] = result_out['no_deposits_out']/24
        result_out['sum_payments_to_debit_turnover'] = result_out['sum_out']/df_turnovers['debit_2y']
        result_out['average_payments_to_debit_turnover'] = result_out['average_out']/df_turnovers['debit_2y']
        
        result2_out = df_out.groupby( 'customer_source_unique_id')[['txn_val_24months', 'txn_val_12months']] .sum() .reset_index()
        result2_out ['cash_out_ratio'] = result2_out ['txn_val_12months']/result2_out['txn_val_24months']
    ### drop year on year values
    result2 = result2.merge(result2_out, how = 'outer', on = 'customer_source_unique_id')
    return result_in, result2, result_out, result_by_instrument_in

In [45]:
# Receipts skewness & kurtosis
def monthly_domestic_stats_features(df, jnumber):
    df_monthly_in = df.loc[df['in_out'].str.lower() == 'in']
    if len(df_monthly_in) == 0:
        monthly_in = pd.DataFrame([[jnumber]+list(np.linspace(0,0,10, dtype=int))]\
                                , columns = ['customer_source_unique_id', 'sum_in', 'average_in', 'median_in',
                                'max_in', 'no_deposits_in', 'variance_in', 'iqr_in', 'skewness_in',
                                'kurtosis_in', 'average_cash_in_to_interq'])
        
    else:
        monthly_in = df_monthly_in.groupby(['customer_source_unique_id', 'year', 'month'], as_index=False).agg(
            
            sum_in = ('txn_amount_base', np.sum),
            average_in = pd.NamedAgg(column='txn_amount_base', aggfunc=lambda x: np.mean([i for i in x if i != 0])),
            median_in = pd.NamedAgg(column='txn_amount_base', aggfunc=lambda x: np.median([i for i in x if i != 0])),
            max_in = ('txn_amount_base', np.max),
            no_deposits_in = ('txn_amount_base', 'count'),
            variance_in = pd.NamedAgg(column='txn_amount_base', aggfunc=lambda x: np.var([i for i in x if i != 0])),
            iqr_in = pd.NamedAgg(column='txn_amount_base', aggfunc=lambda x : iqr([i for i in x if i != 0])),
            skewness_in = pd.NamedAgg(column='txn_amount_base', aggfunc=lambda x: skew([i for i in x if i != 0])),
            kurtosis_in = pd.NamedAgg(column='txn_amount_base', aggfunc=lambda x: kurtosis([i for i in x if i != 0]))
            )
        monthly_in = add_empty_months(monthly_in, all_months, jnumber)
        if 'index' in monthly_in.columns:
            monthly_in.drop('index', axis=1, inplace=True)
    return monthly_in

### 3.3.2 Output

#### 3.3.2.1 Derived attributes

Derived attributes are new attributes that are constructed from one or more existing attributes in
the same record. Describe those here.

#### 3.3.2.2 Generated records

Describe the creation of completely new records.

In [26]:
#Return the dataframe
df_cash = read_clean(Data_Path,r'FakeCashData.csv')[0]
df_cash

,customer_source_unique_id,in_out,posting_date,txn_amount_base,instrument,region,reversal,transaction_location,transaction_location_cntry_cd,source_txn_unique_id,date,month,day,period,txn_val_24months,txn_val_12months,centre_code,PO
0,RUKJ1234567,In,20210723,6789.00,CASH - BRANCH,DOMESTIC,N,118 GLOUCESTER LONDON,GB,R21072300151234AB12C34562001,2021-07-23,7,23,24 month,6789.00,NaN,1234,N
1,RUKJ1234567,In,20210823,10000.00,CASH - BRANCH,DOMESTIC,N,118 GLOUCESTER LONDON,GB,R21082300151234AB12C34562001,2021-08-23,8,23,24 month,10000.00,NaN,1234,N
2,RUKJ1234567,In,20210923,20000.00,CASH - BRANCH,DOMESTIC,N,118 GLOUCESTER LONDON,GB,R21092300151234AB12C34562001,2021-09-23,9,23,24 month,20000.00,NaN,1234,N
3,RUKJ1234567,In,20211023,100.00,CASH - BRANCH,DOMESTIC,N,118 GLOUCESTER LONDON,GB,R21102300151234AB12C34562001,2021-10-23,10,23,24 month,100.00,NaN,1234,N
4,RUKJ1234567,In,20211123,2344.00,CASH - BRANCH,DOMESTIC,N,118 GLOUCESTER LONDON,GB,R21112300151234AB12C34562001,2021-11-23,11,23,24 month,2344.00,NaN,1234,N
5,RUKJ1234567,Out,20211223,9875.00,CASH - CHANNELS,DOMESTIC,N,POST OFFICE LONDON,NaN,R21122300151602AB12C34562001,2021-12-23,12,23,24 month,9875.00,NaN,1602,N
6,RUKJ1234567,Out,20220119,2134.00,CASH - CHANNELS,DOMESTIC,N,POST OFFICE SHEFFIELD,NaN,R22011900151602AB12C34562001,2022-01-19,1,19,24 month,2134.00,NaN,1602,N
7,RUKJ1234567,Out,20220704,1600.00,CASH - CHANNELS,DOMESTIC,N,POST OFFICE LONDON,NaN,R22011900151602AB12C34562001,2022-07-04,7,4,12 month,NaN,1600.00,1602,N
8,RUKJ1234567,In,20220119,15.12,CASH - CHANNELS,DOMESTIC,N,POST OFFICE LONDON,NaN,R22011900151602AB12C34562001,2022-01-19,1,19,24 month,15.12,NaN,1602,N
9,RUKJ1234567,In,20220222,198.76,CASH - BRANCH,DOMESTIC,N,41 HIGH STREET CANTEBURY,GB,R22022200154321AB12C34562001,2022-02-22,2,22,24 month,198.76,NaN,4321,N


In [27]:
#Return the jnumber
jnumber = read_clean(Data_Path,r'FakeCashData.csv')[1]
jnumber

'RUKJ1234567'

read_clean function
transaction_location_cntry_cd
vs
sample_data
transaction_location_cntry_code



In [28]:
result_in = basic_stats_features(df_cash, jnumber)[0]
result_in

,customer_source_unique_id,sum_in,average_in,median_in,max_in,no_deposits_in,variance_in,iqr_in,skewness_in,kurtosis_in,average_cash_in_to_interq
0,RUKJ1234567,127884.19,7992.761875,5000.0,41962.34,16,1.043198e+08,8236.0,2.259428,4.869945,0.970466


In [29]:
result_out = basic_stats_features(df_cash, jnumber)[1]
result_out

,customer_source_unique_id,sum_out,average_out,median_out,max_out,no_deposits_out,variance_out,iqr_out,skewness_out,kurtosis_out
0,RUKJ1234567,19855.4,2836.485714,2000.0,9875.0,7,2.293737e+07,4767.2,0.121123,-0.869845


In [30]:
result2 = basic_stats_features(df_cash, jnumber)[2]
result2

,customer_source_unique_id,txn_val_24months_x,txn_val_12months_x,cash_in_ratio,txn_val_24months_y,txn_val_12months_y,cash_out_ratio
0,RUKJ1234567,66045.85,61838.34,0.936294,12009.0,17846.4,1.486085


In [31]:
# result3
geographic_features(df_cash, jnumber)[0]

,customer_source_unique_id,PO_CDM_cash_pct,PO_CDM_cash_only
0,RUKJ1234567,0.355292,0


In [32]:
# result4
geographic_features(df_cash, jnumber)[1]

,customer_source_unique_id,significant_branches_count,average_deposit_sig_branches
0,RUKJ1234567,5,16489.594


In [33]:
# result8
geographic_features(df_cash, jnumber)[2]

,customer_source_unique_id,branch_changes_yoy
0,RUKJ1234567,5


In [34]:
# result9
geographic_features(df_cash, jnumber)[3]

,customer_source_unique_id,distance,distanceYoY,deposit_accessibility
0,RUKJ1234567,65.892201,5.279064,5


In [35]:
# result5
frequency_features(df_cash, jnumber)[0]

,customer_source_unique_id,var_monthly_sum_in,var_monthly_average_in
0,RUKJ1234567,0,0


In [36]:
# result6
frequency_features(df_cash, jnumber)[1]

,customer_source_unique_id,depvar,mean12,mean24,mean2y,meanYoY
0,RUKJ1234567,0,0,0,0,0


In [38]:
#Return the jnumber
df = read_domestic_clean(Data_Path,r'FakeDomesticData.csv')[0]
df

,customer_source_unique_id,in_out,posting_date,txn_amount_base,instrument,region,reversal,transaction_location,transaction_location_cntry_code,source_txn_unique_id,...,year,month,day,period,txn_val_24months,txn_val_12months,centre_code,sort_code,account_number,account_number_correct_format
0,RUKJ1234567,In,20210723,6789.00,FUND TRANSFER,DOMESTIC,N,,,R21072300151234AB12C34562001,...,2021,7,23,24 month,6789.00,NaN,1234,010101,87654321,Y
1,RUKJ1234567,In,20210823,10000.00,FUND TRANSFER,DOMESTIC,N,,,R21082300151234AB12C34562001,...,2021,8,23,24 month,10000.00,NaN,1234,010101,87654321,Y
2,RUKJ1234567,In,20210923,20000.00,FUND TRANSFER,DOMESTIC,N,,,R21092300151234AB12C34562001,...,2021,9,23,24 month,20000.00,NaN,1234,010101,87654321,Y
3,RUKJ1234567,In,20211023,100.00,FUND TRANSFER,DOMESTIC,N,,,R21102300151234AB12C34562001,...,2021,10,23,24 month,100.00,NaN,1234,010101,87654321,Y
4,RUKJ1234567,In,20211123,2344.00,DEBIT/TOP-UP CARD,DOMESTIC,N,Wiltshire,GB,R21112300151234AB12C34562001,...,2021,11,23,24 month,2344.00,NaN,1234,101010,12345678,Y
5,RUKJ1234567,Out,20211223,9875.00,STANDING ORDER,DOMESTIC,N,,,R21122300151602AB12C34562001,...,2021,12,23,24 month,9875.00,NaN,1602,123456,10101010,Y
6,RUKJ1234567,Out,20220119,2134.00,STANDING ORDER,DOMESTIC,N,,,R22011900151602AB12C34562001,...,2022,1,19,24 month,2134.00,NaN,1602,123456,10101010,Y
7,RUKJ1234567,Out,20220704,1600.00,FUND TRANSFER,DOMESTIC,N,,,R22011900151602AB12C34562001,...,2022,7,4,12 month,NaN,1600.00,1602,654321,10101010,Y
8,RUKJ1234567,In,20220119,15.12,FUND TRANSFER,DOMESTIC,N,,,R22011900151602AB12C34562001,...,2022,1,19,24 month,15.12,NaN,1602,987654,12345678,Y
9,RUKJ1234567,In,20220222,198.76,DEBIT/TOP-UP CARD,DOMESTIC,N,,GB,R22022200154321AB12C34562001,...,2022,2,22,24 month,198.76,NaN,4321,654321,01010101,Y


In [40]:
result_in = basic_domestic_stats_features(df, jnumber)[0]
result_in

,customer_source_unique_id,sum_in,average_in,median_in,max_in,no_deposits_in,variance_in,iqr_in,skewness_in,kurtosis_in,average_receipt_in_to_interq,average_receipts_per_month,sum_receipts_to_credit_turnover,average_receipts_to_credit_turnover
0,RUKJ1234567,127884.19,7992.761875,5000.0,41962.34,16,1.043198e+08,8236.0,2.259428,4.869945,0.970466,0.666667,0.345633,0.021602


In [41]:
result_2 = basic_domestic_stats_features(df, jnumber)[1]
result_2

,customer_source_unique_id,txn_val_24months_x,txn_val_12months_x,receipt_in_ratio,txn_val_24months_y,txn_val_12months_y,cash_out_ratio
0,RUKJ1234567,66045.85,61838.34,0.936294,12009.0,17846.4,1.486085


In [42]:
result_2 = basic_domestic_stats_features(df, jnumber)[2]
result_2

,customer_source_unique_id,sum_out,average_out,median_out,max_out,no_deposits_out,variance_out,iqr_out,skewness_out,kurtosis_out,average_payments_out_to_interq,average_payments_per_month,sum_payments_to_debit_turnover,average_payments_to_debit_turnover
0,RUKJ1234567,19855.4,2836.485714,2000.0,9875.0,7,2.293737e+07,4767.2,0.121123,-0.869845,0.595,0.291667,0.116796,0.016685


In [43]:
result_instrument_dataframe_collection = basic_domestic_stats_features(df, jnumber)[3]
result_instrument_dataframe_collection

,customer_source_unique_id,sum_in,average_in,median_in,max_in,no_deposits_in,variance_in,iqr_in,skewness_in,kurtosis_in,instrument_type,average_receipt_in_to_interq,average_receipts_per_month,percent_instrument_type_of_total
0,RUKJ1234567,89707.46,12815.351429,10000.0,41962.34,7,1.821191e+08,11976.0000,1.204985,0.354320,FUND TRANSFER,1.556016,0.291667,0.701474
0,RUKJ1234567,32275.97,5379.328333,5000.0,9876.00,6,1.276846e+07,5634.9075,0.049676,-1.347857,DEBIT/TOP-UP CARD,0.653148,0.250000,0.252384
0,RUKJ1234567,5900.76,1966.920000,2121.0,3458.76,3,1.652793e+06,1568.8800,-0.178053,-1.500000,STANDING ORDER,0.238820,0.125000,0.046141


In [46]:
monthly_domestic_stats_features(df, jnumber)

C:\Users\mjamesbreen\AppData\Local\Temp\ipykernel_9592\1613488379.py:20: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  skewness_in = pd.NamedAgg(column='txn_amount_base', aggfunc=lambda x: skew([i for i in x if i != 0])),
C:\Users\mjamesbreen\AppData\Local\Temp\ipykernel_9592\1613488379.py:21: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  kurtosis_in = pd.NamedAgg(column='txn_amount_base', aggfunc=lambda x: kurtosis([i for i in x if i != 0]))


,customer_source_unique_id,year,month,sum_in,average_in,median_in,max_in,no_deposits_in,variance_in,iqr_in,skewness_in,kurtosis_in
0,RUKJ1234567,2021,5,0.00,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN
1,RUKJ1234567,2021,6,321.00,321.000,321.000,321.00,1,0.000000e+00,0.000,NaN,NaN
2,RUKJ1234567,2021,7,16646.21,8323.105,8323.105,9857.21,2,2.353478e+06,1534.105,0.0,-2.0
3,RUKJ1234567,2021,8,10000.00,10000.000,10000.000,10000.00,1,0.000000e+00,0.000,NaN,NaN
4,RUKJ1234567,2021,9,20000.00,20000.000,20000.000,20000.00,1,0.000000e+00,0.000,NaN,NaN
5,RUKJ1234567,2021,10,3558.76,1779.380,1779.380,3458.76,2,2.820317e+06,1679.380,0.0,-2.0
6,RUKJ1234567,2021,11,2344.00,2344.000,2344.000,2344.00,1,0.000000e+00,0.000,NaN,NaN
7,RUKJ1234567,2021,12,0.00,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN
8,RUKJ1234567,2022,1,15.12,15.120,15.120,15.12,1,0.000000e+00,0.000,NaN,NaN
9,RUKJ1234567,2022,2,11039.76,5519.880,5519.880,10841.00,2,2.831432e+07,5321.120,0.0,-2.0


---

## 3.4 Integrate data

### 3.4.1 Task

These are methods whereby information is combined from multiple tables or records to create new
records or values.

### 3.4.2 Output

#### 3.4.2.1 Merged data

Merged tables refers to tables that were joined together, or those created by aggregation. Describe
those here.

---

## 3.5 Format data

### 3.5.1 Task

Formatting transformations refer to primarily syntactic modifications made to the data that do
not change its meaning, but might be required by the modeling tool.

### 3.5.2 Output

#### 3.5.2.1  Reformatted data

Some tools have requirements on the order of the attributes, such as the first field being a unique
identifier for each record, or the last field being the outcome field the model is to create via
prediction, or perhpas the modeling tools requires all records to be sorted according to a specific
value. Describe such steps here.